<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/02_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BoWによるテキスト分類
* BoWでも良い性能を出せることが多い。
  * LLMを使って文書分類するときは、BoW+SVMの性能と比較した方が良い。
  * なぜなら、分類性能に大きな差がつかないことも、しばしばあるので。

## 準備

### spaCyの最小限の機能のインストール

* 英語だけ扱えるようになる。

In [ ]:
#!pip install -U spacy
!python -m spacy download en_core_web_sm

### spaCyの日本語形態素解析器のインストール
  * Sudachiという形態素解析器が使えるようになる。

In [ ]:
!python -m spacy download ja_core_news_sm

### datasetsのインストール

In [ ]:
!pip install datasets

### 乱数のシードの設定
* transformersのset_seedを使うと便利。

In [ ]:
from transformers import set_seed

set_seed(0)

## データセットの取得

* ライブドアニュースコーパスのタイトルを使う。

In [ ]:
from datasets import load_dataset

ds = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42,
    shuffle=True,
    trust_remote_code=True,
)

* このsplitsをそのままディスクに保存する。

In [ ]:
ds.save_to_disk("livedoor_ds")

* 保存先のディレクトリを丸ごと`.tar.gz`ファイルに固める。

In [ ]:
!tar zcvf livedoor_ds.tar.gz livedoor_ds

* データセットのsplitsを保存した`.tar.gz`ファイルを解凍する。

In [ ]:
!tar zxvf livedoor_ds.tar.gz

* 解凍して出来上がったディレクトリからデータセットを読み込む。

In [ ]:
from datasets import load_from_disk
ds = load_from_disk("./livedoor_ds")

* 実は記事に重複があるが、気にしないことにする。

In [ ]:
for content in ds["train"]["content"]:
  for content2 in ds["validation"]["content"]:
    if content == content2:
      print(content)
  for content2 in ds["test"]["content"]:
    if content == content2:
      print(content)

* 9種類のクラスラベルを参考までに示しておく。

In [ ]:
category_names = [
    'movie-enter',
    'it-life-hack',
    'kaden-channel',
    'topic-news',
    'livedoor-homme',
    'peachy',
    'sports-watch',
    'dokujo-tsushin',
    'smax',
]

* 内容を少し読んでみる。

In [ ]:
ds["train"]["content"][5705]

## 形態素解析

* 一つのテキストでSudachiによる形態素解析を試してみる。

In [ ]:
import spacy

nlp = spacy.load("ja_core_news_sm")
doc = nlp(ds["train"]["content"][5705])
for token in doc:
  print(token.lemma_, end=" ")

* 訓練データ全体を形態素解析する。
  * TfidfVectorizerの入力として使うので・・・
  * 形態素を空白文字でつないで一つの文字列にしておく。

In [ ]:
from tqdm.auto import tqdm

corpus_train = []
for text in tqdm(ds["train"]["content"]):
  doc = nlp(text)
  text = " ".join([token.lemma_ for token in doc])
  for new_line_char in ["\n", "\r", "\u2028", "\u2029", "\u0085"]:
    text = " ".join([line for line in text.split(new_line_char)])
  corpus_train.append(text)

* ファイルへ書き込む。

In [ ]:
with open('livedoor_content_train.txt', 'w') as f:
  f.write("\n".join(corpus_train) + "\n")

* ファイルから読み込む。

In [ ]:
with open('livedoor_content_train.txt', 'r') as f:
  corpus_train = f.read().splitlines()

In [ ]:
corpus_train[0]

* 検証データとテストデータも形態素解析する。

In [ ]:
corpus_val = []
for text in tqdm(ds["validation"]["content"]):
  doc = nlp(text)
  text = " ".join([token.lemma_ for token in doc])
  for new_line_char in ["\n", "\r", "\u2028", "\u2029", "\u0085"]:
    text = " ".join([line for line in text.split(new_line_char)])
  corpus_train.append(text)

In [ ]:
with open('livedoor_content_val.txt', 'w') as f:
  f.write("\n".join(corpus_val) + "\n")

In [ ]:
with open('livedoor_content_val.txt', 'r') as f:
  corpus_val = f.read().splitlines()

In [ ]:
corpus_test = []
for text in tqdm(ds["test"]["content"]):
  doc = nlp(text)
  text = " ".join([token.lemma_ for token in doc])
  for new_line_char in ["\n", "\r", "\u2028", "\u2029", "\u0085"]:
    text = " ".join([line for line in text.split(new_line_char)])
  corpus_train.append(text)

In [ ]:
with open('livedoor_content_test.txt', 'w') as f:
  f.write("\n".join(corpus_test) + "\n")

In [ ]:
with open('livedoor_content_test.txt', 'r') as f:
  corpus_test = f.read().splitlines()

## TF-IDFベクトルの計算

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10, max_df=0.2)
X_train = vectorizer.fit_transform(corpus_train)

In [ ]:
X_train.shape

# 課題
* ライブドアニュースコーパスを9値分類する分類器を作ろう。
* 注意：データセットの分割の仕方は変えないようにしましょう。

## 実行例

* 交差検証でハイパーパラメータのチューニングをする。

In [ ]:
import numpy as np

corpus = np.array(corpus_train + corpus_val)
labels = np.array(ds["train"]["category"] + ds["validation"]["category"])

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

for min_df in [10, 20, 30]:
  for max_df in [0.2, 0.3, 0.4]:
    vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df)
    for C in 10. ** np.arange(-1, 4):
      scores = []
      skf_split = skf.split(corpus, labels)
      for train_index, val_index in skf_split:
        X_train = vectorizer.fit_transform(corpus[train_index])
        clf = LinearSVC(C=C, dual=False, max_iter=1000, random_state=123)
        clf.fit(X_train, labels[train_index])
        X_val = vectorizer.transform(corpus[val_index])
        score = clf.score(X_val, labels[val_index])
        print(f"\t{score:.3f}", end=" ")
        scores.append(score)
      print(f"\nmean accuracy: {np.array(scores).mean():.3f}", end="")
      print(f" | C={C:.2e} min_df={min_df} max_df={max_df:.3f}")

In [ ]:
vectorizer = TfidfVectorizer(min_df=20)
X = vectorizer.fit_transform(corpus)
clf = LinearSVC(C=10.0, dual=False, max_iter=1000, random_state=123)
clf.fit(X, labels)
X_test = vectorizer.transform(corpus_test)
score = clf.score(X_test, ds["test"]["category"])
print(f"{score:.3f}")
